In [1]:
from scipy.sparse.linalg import svds
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [26]:
data = pd.read_excel("./test_data.xlsx")
data2 = pd.read_excel('./printer_data (1).xlsx')
data3 = pd.read_excel("./user_data.xlsx")

In [27]:
data.head()

,userid,productid,rating
0,1,45,4
1,1,48,2
2,1,49,3
3,1,53,1
4,1,62,5


In [28]:
data=data.drop_duplicates(['userid','productid'], keep='first')

In [29]:
data.head()

,userid,productid,rating
0,1,45,4
1,1,48,2
2,1,49,3
3,1,53,1
4,1,62,5


In [30]:
df_user_ratings =data.pivot_table('rating', index='userid', columns='productid').fillna(0)
df_user_ratings.head()

productid,1,2,3,4,5,6,7,8,9,10,...,5892,5893,5894,5895,5896,5897,5898,5899,5900,5901
userid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [31]:
matrix=df_user_ratings.values

In [32]:
#사용자의 평균 평점
user_ratings_mean=np.mean(matrix, axis=1)

In [33]:
#사용자의 평점을 평균으로 뺌
matrix_user_mean=matrix-user_ratings_mean.reshape(-1,1)

In [47]:
pd.DataFrame(matrix_user_mean, index=df_user_ratings.index, columns=df_user_ratings.columns).head()

productid,1,2,3,4,5,6,7,8,9,10,...,5892,5893,5894,5895,5896,5897,5898,5899,5900,5901
userid,,,,,,,,,,,,,,,,,,,,,
1,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,...,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319,-0.306319
2,-0.266155,-0.266155,-0.266155,3.733845,-0.266155,-0.266155,-0.266155,-0.266155,-0.266155,-0.266155,...,-0.266155,-0.266155,-0.266155,-0.266155,0.733845,-0.266155,-0.266155,-0.266155,-0.266155,-0.266155
3,3.709390,-0.290610,3.709390,-0.290610,3.709390,-0.290610,-0.290610,-0.290610,4.709390,-0.290610,...,-0.290610,-0.290610,-0.290610,-0.290610,-0.290610,-0.290610,-0.290610,-0.290610,-0.290610,-0.290610
4,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,...,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035,-0.302035
5,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,...,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,-0.289539,4.710461


In [48]:
#scipy에서 제공해주는 svd를 사용해 전치 행렬 반환
U, sigma, Vt= svds(matrix_user_mean, k =12)

In [49]:
#0이 포함된 대칭행렬로 변환.
sigma=np.diag(sigma)

In [50]:
#분해한 행렬을 다시 원본 행렬로 복구, 다시 평점 평균을 더해준다.
svd_user_predicted_ratings=np.dot(np.dot(U, sigma), Vt)+user_ratings_mean.reshape(-1,1)

In [51]:
df_svd_preds=pd.DataFrame(svd_user_predicted_ratings, columns=df_user_ratings.columns)

In [52]:
df_svd_preds.head()

productid,1,2,3,4,5,6,7,8,9,10,...,5892,5893,5894,5895,5896,5897,5898,5899,5900,5901
0,-0.076638,-0.449460,-0.364301,-0.011846,-0.622430,0.259490,-0.236961,0.381995,-0.847976,0.088390,...,0.112474,0.413365,-0.324639,0.059901,0.249491,0.518346,0.280390,0.172098,1.337524,0.289549
1,0.065347,0.242463,0.212056,0.076133,0.234047,-0.018368,0.396079,0.320294,0.121494,0.092450,...,0.205161,0.609871,0.145695,0.242047,0.293130,-0.075054,0.327137,0.288056,0.254927,0.385086
2,1.630113,-0.117849,1.921097,-0.271549,1.509666,0.300246,0.380402,0.246645,2.134836,0.459813,...,0.221104,0.240796,0.586952,0.426126,0.357524,1.195855,0.636059,0.491661,-0.257623,-0.408715
3,-0.827456,-0.578931,-1.504709,1.267682,-0.929315,0.013971,0.659885,0.552590,-0.675126,0.679564,...,0.051540,-0.573726,-0.108774,0.237935,0.213637,-0.035690,0.115050,0.205537,-0.154256,0.761913
4,0.301239,-0.251348,-0.415977,0.519615,0.005244,0.290142,0.025864,0.367106,-0.232885,0.180937,...,0.285978,0.316646,0.231322,0.184084,0.310506,0.591950,0.065506,0.239208,0.027612,0.693177


In [76]:
def recommend_product(df_svd_preds, user_id, ori_product_df, ori_ratings_df, num_recommendations=5):
    
    # 최종적으로 만든 df_svd_preds에서 사용자 index에 따라 제품데이터 정렬 -> 제품 평점이 높은 순으로 정렬.
    sorted_user_predictions = df_svd_preds.iloc[user_id].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아냄. 
    user_data = ori_ratings_df[ori_ratings_df.userid == 'userid'+ str(user_id)]
    
    # 위에서 뽑은 user_data와 원본 제품 데이터를 합침. 
    user_history = user_data.merge(ori_product_df, on = 'productid').sort_values(['rating'], ascending=False)
    
    # 원본 제품 데이터에서 사용자가 구매한 제품 데이터를 제외한 데이터를 추출
    recommendations = ori_product_df[~ori_product_df['productid'].isin(user_history['productid'])]
    # 사용자의 제품 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합침. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'productid')
    # 컬럼 이름 바꾸고 정렬해서 return
    
    
                      
    return user_history, recommendations

In [77]:
already_rated, predictions = recommend_product(df_svd_preds, 5, data2, data)

In [78]:
already_rated.head()

,userid,rating,상품명,productid,가격,keyword,productid.1,업종,근무자수,기본 급지용량,내구성


In [79]:
predictions.head(500)

,상품명,productid,가격,keyword,productid.1,업종,근무자수,기본 급지용량,내구성,5
0,ApeosPort 2560,1,20000000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,1,법,87,300,2,-0.065523
1,ApeosPort 3060,2,19500000,흑백 디지털 복합기 잉크젯 포토프린터 컬러 출력 복사 스캔 팩스 인쇄 ...,2,기타,23,1290,3,-0.315861
2,ApeosPort 3560,3,18900000,"흑백 디지털 복합기 잉크젯 컬러 출력 복사 스캔 인쇄 해상도: 4,800d...",3,교육,68,530,2,0.559018
3,ApeosPort C2060,4,18711290,컬러 디지털 복합기 인쇄 복사 스캔 컬러 속도: 23ppm 흑백 속도: 2...,4,행정,25,1200,3,0.172032
4,ApeosPort C2560,5,18299940,컬러 디지털 복합기 인쇄 복사 스캔 팩스 컬러 속도: 23ppm 흑백 속...,5,행정,96,1350,2,-0.054026
...,...,...,...,...,...,...,...,...,...,...
495,Canon IR C3125,518,2660000,"컬러 디지털 복합기 잉크젯 컬러 출력 복사 스캔 인쇄 해상도: 4,800d...",518,기타,18,690,2,0.328961
496,Canon IR C3125,519,2660000,컬러 디지털 복합기 인쇄 복사 스캔 컬러 속도: 23ppm 흑백 속도: 2...,519,은행,92,1200,2,-0.143020
497,Canon IR C3125,520,2659950,컬러 디지털 복합기 인쇄 복사 스캔 팩스 컬러 속도: 23ppm 흑백 속...,520,은행,11,1350,3,0.620086
498,Canon IR C3125,521,2654000,컬러 디지털 복합기 인쇄 복사 스캔 팩스 컬러 속도: 38ppm 흑백 속...,521,사무,72,620,4,-0.066097


In [44]:
df = pd.DataFrame(columns=data2.index ,index=data3.userid)

In [45]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,5891,5892,5893,5894,5895,5896,5897,5898,5899,5900
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
for i in range(1,5901):
    already_rated, predictions = recommend_product(df_svd_preds, i, data2, data)
    df[j] = predictions[str(i)]

KeyError: '1'